# LangChain Quickstart Sample

This project use the AI Search service to create a vector store for a custom department store data.  We’ll use Retrieval Augmented Generation (RAG), a pattern used in AI which uses Azure Open AI LLM to generate answers with your own data. In addition, we’ll construct prompt template to provide the scope of our dataset, as well as the context to the submit questions. Next, we’ll maintain the state of the QnA by storing the chat history in the prompt. Lastly, to enable the user to ask questions our data in a conversational format, we’ll using Langchain to connect our prompt template with our Azure Open AI LLM.

In [17]:
import os
import openai
from typing import Any, List
#from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from azure.ai.generative.index import get_langchain_retriever_from_index

from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.vectorstores import FAISS

Set credential values needed to authenciate and authorize access to your Azure OpenAI instance.

In [18]:
def setup_credentials():
    # Azure OpenAI credentials
    openai.api_type = os.environ["OPENAI_API_TYPE"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    openai.api_version = os.environ["OPENAI_API_VERSION"]
    openai.api_base = os.environ["OPENAI_API_BASE"]

    # Azure Cognitive Search credentials
    os.environ["AZURE_COGNITIVE_SEARCH_TARGET"] = os.environ["AZURE_AI_SEARCH_ENDPOINT"]
    os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = os.environ["AZURE_AI_SEARCH_KEY"]
    os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = os.environ["AZURE_AI_SEARCH_KEY"]
    os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = os.environ["AZURE_AI_SEARCH_INDEX_NAME"]

Execute the authorize with the given credentials above

In [19]:
setup_credentials()

Here, we're using a helper function to extract our AI Search name from the endpoint URL.

In [20]:
from urllib.parse import urlparse

def get_search_resource_name(srv_url):
    val = urlparse(srv_url)
    return val.netloc
endpoint = get_search_resource_name(os.getenv('AZURE_AI_SEARCH_ENDPOINT'))
resource_name =str(endpoint).split('.', 1)[0]

We'll use LangChain's [AzureCognitiveSearchRetriever](https://api.python.langchain.com/en/latest/retrievers/langchain_community.retrievers.azure_cognitive_search.AzureCognitiveSearchRetriever.html#) library.  This is a useful retriever that seamlessly takes the user's query input and searches the vector database in [Azure AI Search](https://learn.microsoft.com/en-us/azure/search/search-what-is-azure-search) (formerly known as "Azure Cognitive Search") with minimal code.  The retriever uses the MLIndex model to find the most relevant results from the vector database.

In [21]:
from langchain.retrievers import AzureCognitiveSearchRetriever
retriever = AzureCognitiveSearchRetriever(content_key="content", top_k=10, service_name=resource_name)

Construct your prompt by specifying what the chat assistant does as well as the scope and domain of topics it can provide information for.  In addition, define any constraints, restrictions or bounderies on how the prompt should behave.  The prompt template includes action the prompt can take; chat history of the system/user dialogue; context of the chat conversation.

In [22]:
template = """
System:
You are an AI assistant helping users with queries related to outdoor outdooor/camping gear and clothing.
Use the following pieces of context to answer the questions about outdoor/camping gear and clothing as completely, correctly, and concisely as possible.

---

{chat_history}

------

{context}

------


Question: {question}

Answer:"
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=[
        "context",
        "chat_history",
        "question"
    ],
)

Initialize the AzureChatOpenAI instance with your Azure Open AI model (gpt-35-turbo) and deployment.  The temperature value ranges from 0 to 1.  Value closer to 0 denotes how specific you want the response to be; and a value closer to 1 denoter how random you want the responses to be generated.

In [24]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
    model_name=os.environ["AZURE_OPENAI_CHAT_MODEL"],
    temperature=0.7
)

ConversationBufferMemory is a straigh-forward what for storing conversation history of a chat. The function takes the input and output of the chat and stores it in memory. The `chat_history` field from the prompt gets populated by memory.  The`memory_key` is where you specify which key to store in memory.

In [25]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

Now, you are ready to invoke the type of langchain you want to use.  **ConversationalRetrievalChain** connects the Azure Open AI LLM model, retriever, prompt template and chat memory in order to search the AI Search database to retrieve the most relevant response.  To activate the instance you need your LLM model to retrieve response, the promt template rules, and chat history.  Verbose is set to False to not display the prompt structure with the response values.

In [26]:
qna_chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=retriever,
                                           condense_question_prompt=prompt_template,
                                           #return_source_documents=True,
                                           verbose=False,
                                           memory=memory)

Let's test the chat with a question!

In [27]:
#Enter a question about outdoor gear and clothing
inquiry = "Which of your sleeping bags are polyester?"
response = qna_chain({"question": inquiry})
response['answer']

'The CozyNights Sleeping Bag (item_number: 7) and the MountainDream Sleeping Bag (item_number: 14) are both made of polyester.'